- 自动求导机制通过有向无环图（directed acyclic graph ，DAG）实现
- 在DAG中，记录数据（对应tensor.data）以及操作（对应tensor.grad_fn）
- 操作在pytorch中统称为Function，如加法、减法、乘法、ReLU、conv、Pooling等，统统是Function

- torch.autograd.backward(tensors, grad_tensors=None, retain_graph=None, create_graph=False, grad_variables=None, inputs=None)
- 用于对张量进行反向传播计算梯度，参数包括张量列表、梯度张量列表、是否保留计算图、是否创建计算图、梯度变量列表和输入变量列表
- tensors (Sequence[Tensor] or Tensor) – 用于求导的张量
- grad_tensors (Sequence[Tensor or None] or Tensor, optional) – 雅克比向量积中使用，详细作用请看代码演示
- retain_graph (bool, optional) – 是否需要保留计算图。pytorch的机制是在方向传播结束时，计算图释放以节省内存。大家可以尝试连续使用loss.backward()，就会报错。如果需要多次求导，则在执行backward()时，retain_graph=True
- create_graph (bool, optional) – 是否创建计算图，用于高阶求导
- 输入（Sequence[Tensor]或 Tensor，可选）- 与之相关的梯度将累积到.grad 中,所有其他 Tensor 将被忽略;如果不提供，梯度将累积到用于计算 attr::tensors 的所有叶 Tensor 中

In [ ]:
#retain_grad参数的使用
import torch

w = torch.tensor([1.],requires_grad = True)
x = torch.tensor([2.],requires_grad = True)

a = torch.add(w,x)
b = torch.add(w,1)
y = torch.mul(a,b)

y.backward(retain_graph = True)
print(w.grad)
# 由于 PyTorch 梯度是累加的
# 第二次反向传播的结果会加到第一次的结果上
y.backward()
print(w.grad)

tensor([5.])
tensor([10.])


In [2]:
#### retain_grad = False
w = torch.tensor([1.],requires_grad = True)
x = torch.tensor([2.],requires_grad = True)

a = torch.add(w,x)
b = torch.add(w,1)
y = torch.mul(a,b)

y.backward(retain_graph = False)
print(w.grad)
y.backward()
print(w.grad)

tensor([5.])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
# grad_tensors的使用

w = torch.tensor([1.],requires_grad = True)
x = torch.tensor([2.],requires_grad = True)

a = torch.add(w,x)
b = torch.add(w,1)

y0 = torch.mul(a,b)       # y0 = (x+w) * (w+1)    dy0/dw = 2w + x + 1
y1 = torch.add(a,b)       # y1 = (x+w) + (w+1)    dy1/dw = 2

loss = torch.cat([y0,y1],dim = 0)      # [y0, y1]
print(loss)
grad_tensors  = torch.tensor([1.,2.])

# Tensor.backward中的 gradient 传入 torch.autograd.backward()中的grad_tensors
loss.backward(gradient = grad_tensors)

# w =  1* (dy0/dw)  +   2*(dy1/dw)
# w =  1* (2w + x + 1)  +   2*(w)
# w =  1* (5)  +   2*(2)
# w =  9

print(w.grad)

tensor([6., 5.], grad_fn=<CatBackward0>)
tensor([9.])


- torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False)
- 功能：计算outputs对inputs的导数
- 主要参数：
- outputs (sequence of Tensor) – 用于求导的张量，如loss
- inputs (sequence of Tensor) – 所要计算导数的张量
- grad_outputs (sequence of Tensor) – 雅克比向量积中使用
- retain_graph (bool, optional) – 是否需要保留计算图。pytorch的机制是在方向传播结束时，计算图释放以节省内存。大家可以尝试连续使用loss.backward()，就会报错。如果需要多次求导，则在执行backward()时,retain_graph=True
- create_graph (bool, optional) – 是否创建计算图，用于高阶求导
- allow_unused (bool, optional) – 是否需要指示，计算梯度时未使用的张量是错误的

In [ ]:
x = torch.tensor([3.],requires_grad=True)
y = torch.pow(x,2)

# 一阶导数
grad_1 = torch.autograd.grad(y,x,create_graph = True)  # grad_1 = dy/dx = 2x = 2 * 3 = 6
print(grad_1)

#二阶导数
grad_2  = torch.autograd.grad(grad_1[0],x)      # grad_2 = d(dy/dx)/dx = d(2x)/dx = 2
print(grad_2)

(tensor([6.], grad_fn=<MulBackward0>),)
(tensor([2.]),)


- torch.autograd.Function
- 有的时候，想要实现自己的一些操作（op），如特殊的数学函数、pytorch的module中没有的网络层，那就需要自己写一个Function，在Function中定义好forward的计算公式、backward的计算公式，然后将这些op组合到模型中，模型就可以用autograd完成梯度求取
- 这个概念还是很抽象，平时用得不多，但是自己想要自定义网络时，常常需要自己写op，那么它就很好用了，为了让大家掌握自定义op——Function的写法，特地从多处收集了四个案例，大家多运行代码体会Function如何写

In [7]:
# eg1
from torch.autograd import Function
class Exp(Function):                    # 此层计算e^x

    @staticmethod
    def forward(ctx, i):                # 模型前向
        result = i.exp()
        ctx.save_for_backward(result)   # 保存所需内容，以备backward时使用，所需的结果会被保存在saved_tensors元组中；此处仅能保存tensor类型变量，若其余类型变量（Int等），可直接赋予ctx作为成员变量，也可以达到保存效果
        return result

    @staticmethod
    def backward(ctx, grad_output):     # 模型梯度反传
        result, = ctx.saved_tensors     # 取出forward中保存的result
        return grad_output * result     # 计算梯度并返回

# 尝试使用
x = torch.tensor([1.], requires_grad=True)  # 需要设置tensor的requires_grad属性为True，才会进行梯度反传
ret = Exp.apply(x)                          # 使用apply方法调用自定义autograd function
print(ret)                                  # tensor([2.7183], grad_fn=<ExpBackward>)
ret.backward()                              # 反传梯度
print(x.grad)                               # tensor([2.7183])

tensor([2.7183], grad_fn=<ExpBackward>)
tensor([2.7183])


In [8]:
# eg2
class GradCoeff(Function):       
       
    @staticmethod
    def forward(ctx, x, coeff):                 # 模型前向
        ctx.coeff = coeff                       # 将coeff存为ctx的成员变量
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):             # 模型梯度反传
        return ctx.coeff * grad_output, None    # backward的输出个数，应与forward的输入个数相同，此处coeff不需要梯度，因此返回None

# 尝试使用
x = torch.tensor([2.], requires_grad=True)
ret = GradCoeff.apply(x, -0.1)                  # 前向需要同时提供x及coeff，设置coeff为-0.1
ret = ret ** 2                          
print(ret)                                      # tensor([4.], grad_fn=<PowBackward0>)
ret.backward()  
print(x.grad)                                   # tensor([-0.4000])，梯度已乘以相应系数

tensor([4.], grad_fn=<PowBackward0>)
tensor([-0.4000])


In [9]:
# eg3 
import math

class LegendrePolynomial3(Function):
    @staticmethod
    def forward(ctx, x):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        y = 0.5 * (5 * x ** 3 - 3 * x)
        ctx.save_for_backward(x)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        ret, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * ret ** 2 - 1)

a, b, c, d = 1, 2, 1, 2 
x = 1
P3 = LegendrePolynomial3.apply
y_pred = a + b * P3(c + d * x)
print(y_pred)


127.0


In [10]:
# eg4 手动实现2d卷积
from torch.autograd.function import once_differentiable
import torch.nn.functional as F

def convolution_backward(grad_out, X, weight):
    """
    将反向传播功能用函数包装起来，返回的参数个数与forward接收的参数个数保持一致，为2个
    """
    grad_input = F.conv2d(X.transpose(0, 1), grad_out.transpose(0, 1)).transpose(0, 1)
    grad_X = F.conv_transpose2d(grad_out, weight)
    return grad_X, grad_input

class MyConv2D(torch.autograd.Function):
    @staticmethod
    def forward(ctx, X, weight):
        ctx.save_for_backward(X, weight)

        # ============== step1: 函数功能实现 ==============
        ret = F.conv2d(X, weight) 
        # ============== step1: 函数功能实现 ==============
        return ret

    @staticmethod
    def backward(ctx, grad_out):
        X, weight = ctx.saved_tensors
        return convolution_backward(grad_out, X, weight)
      
weight = torch.rand(5, 3, 3, 3, requires_grad=True, dtype=torch.double)
X = torch.rand(10, 3, 7, 7, requires_grad=True, dtype=torch.double)
# gradcheck 会检查你实现的自定义操作的前向传播 (forward) 和反向传播 (backward) 方法是否正确计算了梯度。
# 如果返回 True，则表示梯度检查通过，即自定义操作的梯度计算与数值近似梯度之间的一致性在允许的误差范围内；
# 如果返回 False，则说明存在不匹配，需要检查和修正自定义操作的反向传播逻辑。
print("梯度检查: ", torch.autograd.gradcheck(MyConv2D.apply, (X, weight))) # gradcheck 功能请自行了解，通常写完Function会用它检查一下
y = MyConv2D.apply(X, weight)
label = torch.randn_like(y)
loss = F.mse_loss(y, label)

print("反向传播前，weight.grad: ", weight.grad)
loss.backward()
print("反向传播后，weight.grad: ", weight.grad)

梯度检查:  True
反向传播前，weight.grad:  None
反向传播后，weight.grad:  tensor([[[[1.1148, 1.0969, 1.1268],
          [1.0943, 1.1037, 1.0788],
          [1.0956, 1.1178, 1.1047]],

         [[1.0829, 1.0741, 1.1091],
          [1.1102, 1.1198, 1.1556],
          [1.1514, 1.1903, 1.2224]],

         [[1.1321, 1.1251, 1.1427],
          [1.1727, 1.1364, 1.1209],
          [1.1295, 1.0880, 1.0976]]],


        [[[1.2814, 1.3066, 1.3202],
          [1.3005, 1.3455, 1.2953],
          [1.2799, 1.3291, 1.2988]],

         [[1.2950, 1.3023, 1.3131],
          [1.3244, 1.2974, 1.3382],
          [1.3832, 1.4036, 1.4346]],

         [[1.3653, 1.3588, 1.3755],
          [1.3481, 1.3425, 1.3514],
          [1.3506, 1.3239, 1.2739]]],


        [[[1.0941, 1.1083, 1.1338],
          [1.0983, 1.1176, 1.0827],
          [1.0873, 1.1389, 1.0898]],

         [[1.1026, 1.0919, 1.1319],
          [1.1288, 1.1071, 1.1430],
          [1.1660, 1.1792, 1.2043]],

         [[1.1642, 1.1435, 1.1497],
          [1.1875, 1.12

- autograd相关的知识点
- 知识点一：梯度不会自动清零
- 知识点二： 依赖于叶子结点的结点，requires_grad默认为True
- 知识点三： 叶子结点不可执行in-place
- 知识点四： detach 的作用
- 知识点五： with torch.no_grad()的作用

In [11]:
# 知识点1 : 梯度不会自动清零
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

for i in range(4):
    a = torch.add(w, x)
    b = torch.add(w, 1)
    y = torch.mul(a, b)
    y.backward()
    print(w.grad)           # 梯度不会自动清零，会累加,通常采用optimizer.zero_grad()清零梯度

# w.grad.zero_()  # 清零梯度

tensor([5.])
tensor([10.])
tensor([15.])
tensor([20.])


In [12]:
# 知识点2 ： 依赖于叶子结点的结点，requires_grad默认为True
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)
print(a.requires_grad, b.requires_grad, y.requires_grad)  # True True True
print(a.is_leaf, b.is_leaf, y.is_leaf)  

True True True
False False False


In [14]:
# 知识点3 ： 叶子张量不可以执行in-place操作
# 叶子结点不可执行in-place，因为计算图的backward过程都依赖于叶子结点的计算，
# 可以回顾计算图当中的例子，所有的偏微分计算所需要用到的数据都是基于w和x（叶子结点），因此叶子结点不允许in-place操作
a = torch.ones((1,))
print(id(a),a)

a = a + torch.ones((1,))
print(id(a),a)

a += torch.ones((1,))
print(id(a),a)

w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

w.add_(1)

y.backward()

129936892929968 tensor([1.])
129936893835488 tensor([2.])
129936893835488 tensor([3.])


RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [22]:
# 知识点4 ： detach的作用
# 通过以上知识，我们知道计算图中的张量是不能随便修改的，否则会造成计算图的backward计算错误，
# 那有没有其他方法能修改呢？当然有，那就是detach()

# detach的作用是：从计算图中剥离出“数据”，并以一个新张量的形式返回，并且新张量与旧张量共享数据，简单的可理解为做了一个别名。 
# 请看下例的w，detach后对w_detach修改数据，w同步地被改为了999

w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()

w_detach = w.detach()
w_detach.data[0] = 999
print(w)

tensor([999.], requires_grad=True)


In [ ]:
# 知识点5 ：with torch.no_grad()的作用
# autograd自动构建计算图过程中会保存一系列中间变量，以便于backward的计算，这就必然需要花费额外的内存和时间
# 而并不是所有情况下都需要backward，例如推理的时候，因此可以采用上下文管理器——torch.no_grad()来管理上下文，
# 让pytorch不记录相应的变量，以加快速度和节省空间